## Regression Week 1: Simple Linear Regression

In this notebook we will use data on house sales in King County to predict house prices using simple (one input) linear regression. You will:
* Use graphlab SArray and SFrame functions to compute important summary statistics
* Write a function to compute the Simple Linear Regression weights using the closed form solution
* Write a function to make predictions of the output given the input feature
* Turn the regression around to predict the input given the output
* Compare two different models for predicting house prices

In this notebook you will be provided with some already complete code as well as some code that you should complete yourself in order to answer quiz questions. The code we provide to complte is optional and is there to assist you with solving the problems but feel free to ignore the helper code and write your own.

### Fire up graphlab create

In [1]:
import graphlab

### Load house sales data
Dataset is from house sales in King Country, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('../week1/kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to jaekeunprk@gmail.com and will expire on March 15, 2019.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1544762022.log


### Split data into training and testing
We use seed=0 so that everyone running this notebook gets the same results. In practice you may set a random seed (or let GraphLab Create pick a random seed for you).

In [3]:
train_data, test_data = sales.random_split(.8, seed=0)

### Useful SFrame summary functions
In order to make use of the closed form solution as well as take advantage of graphlab's built in functions we will review some important ones. In particular:
* Computing the sum of an SArray
* Computing the arithmetic average (mean) of an SArray
* multiplying SArrays by constants
* multiplying SArrays by other SArrays

In [5]:
# Let;s compute the mean of the House Prices in King County in 2 different ways.
prices = sales['price'] # extract the price column of the sales SFrame -- this is now an SArray

# recall that the arithmetic average (the mean) is the sum of the prices divided by the total number of houses:
sum_prices = prices.sum()
num_houses = prices.size() # when prices is an SArray .size() returns its length
avg_price_1 = sum_prices / num_houses
avg_price_2 = prices.mean() # if you just want the average, the .mean() function
print "average price via method 1: " + str(avg_price_1)
print "average price via method 2: " + str(avg_price_2)

average price via method 1: 540088.141905
average price via method 2: 540088.141905


As we see we get the same answer both ways

In [6]:
# if we want to multiply every price by 0.5 it's a simple as:
half_price = 0.5*prices
# Let's compute the sum of squares of price. We can multiply two SArrays of the same length elementwise also with *
prices_sqaured = prices*prices
sum_prices_sqaured = prices_sqaured.sum() # price_sqaured is an SArray of the sqaures and we wnat to add them up.
print "the sum of price squared is: " + str(sum_prices_sqaured)

the sum of price squared is: 9.21732513355e+15


### Build a generic simple linear regression function
Armed with these SArray functions we can use the closed form solution found from lecture to compute the slope and intercept for a simple linear regression on observations stored as SArrays: input_feature, output.

Complete the following function (or write your own) to compute the simple linear regression slope and intercept:

In [11]:
def simple_linear_regression(X, y):
    # compute the sum of input_feature and output
    sum_X = X.sum()
    sum_y =y.sum()
    
    # compute the product of the output and the input_feature and its sum
    sum_X_y = sum(X*y)
    
    # compute the squared value of the input_feature and its sum
    sum_X_squared = sum(X*X)
    
    # use the formula for the slope
    numerator = sum_X_y - sum(y)*sum(X) / y.size()
    denominator = sum_X_squared - sum(X)*sum(X) / y.size()
    slope = numerator / denominator
    
    # use the formula for the intercept
    intercept = sum(y) / y.size() - slope * sum(X) / y.size()
    
    return (intercept, slope)

We can test that our function works by passing it something where we know the answer. In particular we can generate a feature and then put the output exactly on a line: output = 1 + 1\*input_feature then we know both our slope and intercept should be 1

In [23]:
test_feature = graphlab.SArray(range(5))
test_output = graphlab.SArray(1 + 1*test_feature)
(test_intercept, test_slope) = simple_linear_regression(test_feature, test_output)
print "Intercept: " + str(test_intercept)
print "Slope: " + str(test_slope)

Intercept: 1.0
Slope: 1.0


In [37]:
sqft_X = train_data['sqft_living']
y = train_data['price']
sqft_intercept, sqft_slope = simple_linear_regression(sqft_X, y)
print "Intercept: " + str(sqft_intercept)
print "Slpoe: " + str(sqft_slope)

Intercept: -47116.0765749
Slpoe: 281.958838568


### Predicting Values
Now that we have the model parameters: intercept & slope we can make predictions. Using SArrays it's easy to multiply an SArray by a constant and add a constant value. Complete the following function to return the predicted output given the input_feature, slope and intercept:

In [18]:
def get_regression_prediction(X, intercept, slope):
    # calcuate the predicted values:
    preds = intercept + slope * X

    return preds

Now that we can calculate a prediction given the slope and intercept let's make a prediction. Use (or alter) the following to find out the estimated price for a house with 2650 squarefeet according to the squarefeet model we estiamted above.

**Quiz Question: Using your Slope and Intercept from (4), What is the predicted price for a house with 2650 sqft?**

In [20]:
get_regression_prediction(2650, sqft_intercept, sqft_slope)

700074.8456294581

### Residual Sum of Squares
Now that we have a model and can make predictions let's evaluate our model using Residual Sum of Squares (RSS). Recall that RSS is the sum of the squares of the residuals and the residuals is just a fancy word for the difference between the predicted output and the true output. 

Complete the following (or write your own) function to compute the RSS of a simple linear regression model given the input_feature, output, intercept and slope:

In [28]:
def get_residual_sum_of_squares(X, y, intercept, slope):
    # First get the predictions
    preds = intercept + slope * X
    
    # then compute the residuals (since we are squaring it doesn't matter which order you subtract)
    residuals = y - preds
    
    # square the residuals and add them up
    RSS = sum(residuals*residuals)
    
    return(RSS)

Let's test our get_residual_sum_of_squares function by applying it to the test model where the data lie exactly on a line. Since they lie exactly on a line the residual sum of squares should be zero!

In [29]:
print get_residual_sum_of_squares(test_feature, test_output, test_intercept, test_slope)

0.0


Now use your function to calculate the RSS on training data from the squarefeet model calculated above.

**Quiz Question: According to this function and the slope and intercept from the squarefeet model What is the RSS for the simple linear regression using squarefeet to predict prices on TRAINING data?**

In [32]:
sqft_rss = get_residual_sum_of_squares(sqft_X, y, sqft_intercept, sqft_slope)
print "The RSS of predicting Prices based on Square Feet is : " + str(sqft_rss)

The RSS of predicting Prices based on Square Feet is : 1.20191835632e+15


### Predict the squarefeet given price
What if we want to predict the squarefoot given the price? Since we have an equation y = a + b\*x we can solve the function for x. So that if we have the intercept (a) and the slope (b) and the price (y) we can solve for the estimated squarefeet (x).

Complete the following function to compute the inverse regression estimate, i.e. predict the input_feature given the output.

In [34]:
def inverse_regression_predictions(y, intercept, slope):
    # solve output = intercept + slope*input_feature for input_feature.
    # Use this equation to compute the inverse predictions:
    estimated_X = (y - intercept) / slope
    
    return estimated_X

Now that we have a function to compute the squarefeet given the price from our simple regression model let's see how big we might expect a house that costs $800,000 to be.

**Quiz Question: According to this function and the regression slope and intercept from (3) what is the estimated square-feet for a house costing $800,000?**

In [36]:
my_house_price = 800000
estimated_sqft = inverse_regression_predictions(my_house_price, sqft_intercept, sqft_slope)
print "The estimated squarefeet for a house worth $%.2f is %d" % (my_house_price, estimated_sqft)

The estimated squarefeet for a house worth $800000.00 is 3004


### New Model: estimate prices from bedrooms
We have made one model for predicting house prices using squarefeet, but there are many other features in the sales SFrame. 
Use your simple linear regression function to estimate the regression parameters from predicting Prices based on number of bedrooms. Use the training data!

In [41]:
# Estimate the slope and intercept for predicting 'price' based on 'bedrooms'


bedrooms_X = train_data['bedrooms']
y = train_data['price']

bedrooms_intercept, bedrooms_slope = simple_linear_regression(bedrooms_X, y)
print "intercpet : " + str(bedrooms_intercept)
print "slope : " + str(bedrooms_slope)

get_regression_prediction(5000, bedrooms_intercept, bedrooms_slope)

bedrooms_rss = get_residual_sum_of_squares(bedrooms_X, y, bedrooms_intercept, bedrooms_slope)
print "The RSS of predicting Prices based on Bedrooms is : " + str(bedrooms_rss)

intercpet : 109473.180469
slope : 127588.952175
The RSS of predicting Prices based on Bedrooms is : 2.14324449423e+15


### Test your Linear Regression Algorithm
Now we have two models for predicting the price of a house. How do we know which one is better? Calculate the RSS on the TEST data (remember this data wasn't involved in learning the model). Compute the RSS from predicting prices using bedrooms and from predicting prices using squarefeet.

**Quiz Question: Which model (square feet or bedrooms) has lowest RSS on TEST data? Think about why this might be the case.**

In [42]:
# Compute RSS when using bedrooms on TEST data:
bedrooms_test_X = test_data['bedrooms']
test_y = test_data['price']

bedrooms_test_rss = get_residual_sum_of_squares(bedrooms_test_X, test_y, bedrooms_intercept, bedrooms_slope)
print "The RSS of predicting Prices based on Bedrooms is : " + str(bedrooms_test_rss)

The RSS of predicting Prices based on Bedrooms is : 4.93364582868e+14


In [44]:
# Compute RSS when using squarefeet on TEST data:
sqft_test_X = test_data['sqft_living']
test_y = test_data['price']

sqft_test_rss = get_residual_sum_of_squares(sqft_test_X, test_y, sqft_intercept, sqft_slope)
print "The RSS of predicting Prices based on Square Feet is : " + str(sqft_test_rss)

The RSS of predicting Prices based on Square Feet is : 2.75402936247e+14
